# Użycie modeli językowych przystosowanych do języka polskiego

Zmiana koncepcji - będę używać modeli, które testowałem na laboratorium
Muszę zweryfikować jakie okno kontekstowe mają te modele, żeby odpowiednio dobrać długość chunków

Na razie myślę że użyję prostego podziełu na chunki o stałej liczbie znaków

Będę tworzyć jeden podział dokumentów więc chunki muszą się zmieścić do każdego modelu
policze średnią liczbę tokenów na znak i zostawię bezpieczny zapas


## Reranker
Zgodnie z dokumentacją `polish-reranker-roberta-v3` wspiera kontekst 8192 tokenów

## Retrieval

### BM25
Nie ma ograniczenia długości kontekstu

### `mmlw-retrieval-roberta-base`

Okno kontekstu 512 tokenów

Dla przykładowego dokumentu 0.24 tokena na znak, 1.71 tokena na słowo

## Genrator

### Bielik
W karcie modelu jest podany ogromny rozmiar kontekstu - nie powinno być problemu

## Przykładowy dokument

In [5]:
from pathlib import Path

example_doc_path = Path(
    "../data/raw/notes/inzynieria-uczenia-maszynowego/02-zadania-modelowania.md"
)
example_doc = example_doc_path.read_text()
example_doc

'# Definiowanie zadań modelowania\n\n## Kluczowe elementy\n* Jakie jest zadanie biznesowe / domenowe\n\t* czy je dostatecznie dobrze rozumiemy\n\t* jakie są ograniczenia (czasowe, zasobów, formalne, ...)\n* Jakie są zadania modelowania\n\t* czy są odpowiednie dane do budowy modeli\n\t* czy rozumiemy proces, który je wygenerował\n* Jakie są kryteria sukcesu\n\t* na poziomie zadań modelowania (analityczne)\n\t* biznesowe\n\n## Formułowanie zadania biznesowego\n* Potrzebny jest ekspert domenowy\n\t* np. product owner w metodologiach zwinnych\n\t* dobrze rozumie zagadnienie, którym mamy się zająć i wynikające z niego potrzeby\n\t* zna specyfikę pracy nad modelowaniem danych\n\t* jest osobą decyzyjną w zakresie projektu - wspólne ustalenia są wiążące\n* Analizujemy kontekst w jakim występuje potrzeba biznesowa\n\t* jaka jest obecna sytuacja\n\t* co ma zostać\xa0wprowadzone / zmienione\n\t* jakie właściwości / cechy powinno mieć docelowe rozwiązanie\n\t* jakie są założenia, oczekiwania, ogra

In [7]:
len(example_doc)

7882

In [20]:
n_example_doc_words = len(example_doc.split())
n_example_doc_words

1090

## bi-enkoder

In [1]:
from sentence_transformers import SentenceTransformer

bi_encoder_model = SentenceTransformer("sdadas/mmlw-retrieval-roberta-base")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/75.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
bi_encoder_model.max_seq_length

512

In [4]:
bi_encoder_model.tokenizer

RobertaTokenizerFast(name_or_path='sdadas/mmlw-retrieval-roberta-base', vocab_size=50001, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50000: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [21]:
bi_encoder_tokens = bi_encoder_model.tokenizer.tokenize(example_doc)
print(f"Średnio tokenów na znak: {len(bi_encoder_tokens) / len(example_doc):.4f}")
print(f"Średnio tokenów na słowo: {len(bi_encoder_tokens) / n_example_doc_words:.4f}")
print(f"Liczba tokenów: {len(bi_encoder_tokens)}")
print("Przykładowe tokeny:")
for token in bi_encoder_tokens[:20]:
    print(token)

Średnio tokenów na znak: 0.2359
Średnio tokenów na słowo: 1.7055
Liczba tokenów: 1859
Przykładowe tokeny:
▁
#
▁Defini
owanie
▁zadań
▁model
owania
▁
##
▁Klucz
owe
▁elementy
▁
*
▁Jakie
▁jest
▁zadanie
▁biznesowe
▁
/
